In [1]:
from bs4 import BeautifulSoup as bs
from getpass import getpass
from IPython.core.display import display, HTML
from lxml import html
from pprint import pprint
from pymongo import MongoClient
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from tqdm import tqdm
import datetime
import json
import math
import numpy as np
import pandas as pd
import pickle
import re
import requests
import requests as rq
import time
import tqdm
import xmltodict, json
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(suppress=False)
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('precision', 2)

In [10]:
"""
1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика 
и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
"""

# Собирает почтовые сообщения, используя новый интерфейс почты
login = input("Введите логин от учетной записи почты: ")
password = print("Введите пароль от учетной записи почты: ")
password = getpass()
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://account.mail.ru/login')

time.sleep(2)
element = driver.find_element_by_xpath("//input[@name='username']")
element.send_keys(login)
element.send_keys(Keys.ENTER)
time.sleep(1)
element = driver.find_element_by_xpath("//input[@name='password']")
element.send_keys(password)
element.send_keys(Keys.ENTER)

time.sleep(10)

for i in range(2):
        time.sleep(1)
        let = driver.find_elements_by_class_name("llc__content")
        actions = ActionChains(driver)

        actions.move_to_element(let[-1])
        actions.perform()

messages = driver.find_elements_by_xpath("//a[@class='llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal']")
print(f"Количество сообщений: {len(messages)}")
links = []
for i in messages:
    links.append(i.get_attribute("href"))
letters = []
for url in links:
    message = {}
    driver.get(url)
    time.sleep(3)
    message['from'] = driver.find_element_by_class_name("letter-contact").text
    message['date'] = driver.find_element_by_class_name("letter__date").text
    message['title'] = driver.find_element_by_class_name("thread__subject").text
    letters.append(message)
df = pd.DataFrame(letters)
display(df)
driver.quit()

Введите логин от учетной записи почты: shagen.madunts@mail.ru
Введите пароль от учетной записи почты: 
········
Количество сообщений: 30


,from,date,title
0,Интернет Магазин Товаров для Дома,"9 июля, 16:58","(МСК) Ваш заказ передан в пункт самовывоза курьерской службы ""Балтийский курьер"""
1,noreply@fcod.nalog.ru,"9 июля, 16:20",6e19b2a57d3e4c54b4f52fbb4a2ec59a ЗГ@ от 20.06.2020: Интернет-обращение из сервиса ЛК2
2,Вселенная порядка,"9 июля, 14:23",Вы оформили заказ 27247 на сайте Вселенная порядка (vselennayaporyadka.ru)
3,ABBYY Online Services,"9 июля, 11:10",Ознакомительный режим FineReader Online
4,ABBYY Online Services,"9 июля, 11:09",Добро пожаловать в FineReader Online
5,GitHub,"8 июля, 18:40",[GitHub] Please verify your device
6,tickets@infotec.ru,"7 июля, 9:53",Ваш заказ N205331 оплачен
7,LOGINOVA Kseniya,"6 июля, 18:44","RE: Re[2]: ипотека Росбанк Дом_Мадунц, Шаген Павлович"
8,parking.spb.ru,"5 июля, 20:15",Успешная регистрация на портале parking.spb.ru
9,parking.spb.ru,"5 июля, 20:14",Подтверждение адреса e-mail для регистрации на портале parking.spb.ru


In [126]:
def to_dataset(db_name, col_name, data):
    client = MongoClient('localhost', 27017)
    db = client[db_name]
    collection_name = f"{col_name}_{datetime.datetime.now().strftime(format='%Y-%m-%d-%H-%M-%S')}"
    collection = db[collection_name]
    for el in data:
        collection.insert_one(el)
    # проверяем
    display(pd.DataFrame(db[collection_name].find( {} )).head(10))

In [12]:
to_dataset('letters', 'mail', letters)

,_id,from,date,title
0,5f0b748eefd9c8a76cd79458,Интернет Магазин Товаров для Дома,"9 июля, 16:58","(МСК) Ваш заказ передан в пункт самовывоза курьерской службы ""Балтийский курьер"""
1,5f0b748eefd9c8a76cd79459,noreply@fcod.nalog.ru,"9 июля, 16:20",6e19b2a57d3e4c54b4f52fbb4a2ec59a ЗГ@ от 20.06.2020: Интернет-обращение из сервиса ЛК2
2,5f0b748eefd9c8a76cd7945a,Вселенная порядка,"9 июля, 14:23",Вы оформили заказ 27247 на сайте Вселенная порядка (vselennayaporyadka.ru)
3,5f0b748eefd9c8a76cd7945b,ABBYY Online Services,"9 июля, 11:10",Ознакомительный режим FineReader Online
4,5f0b748eefd9c8a76cd7945c,ABBYY Online Services,"9 июля, 11:09",Добро пожаловать в FineReader Online
5,5f0b748eefd9c8a76cd7945d,GitHub,"8 июля, 18:40",[GitHub] Please verify your device
6,5f0b748eefd9c8a76cd7945e,tickets@infotec.ru,"7 июля, 9:53",Ваш заказ N205331 оплачен
7,5f0b748eefd9c8a76cd7945f,LOGINOVA Kseniya,"6 июля, 18:44","RE: Re[2]: ипотека Росбанк Дом_Мадунц, Шаген Павлович"
8,5f0b748eefd9c8a76cd79460,parking.spb.ru,"5 июля, 20:15",Успешная регистрация на портале parking.spb.ru
9,5f0b748eefd9c8a76cd79461,parking.spb.ru,"5 июля, 20:14",Подтверждение адреса e-mail для регистрации на портале parking.spb.ru


In [ ]:
"""
2) Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. 
Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары
"""

In [122]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')
driver = webdriver.Chrome('./chromedriver', options=chrome_options)

driver.get('https://www.mvideo.ru/')
count = 0

In [123]:
count = 5 #тут нужно получить явно число, но поставим пока 5)

In [124]:
goods = []
for i in range(1, count):
    hp = driver.find_elements_by_xpath("//div[@data-init='ajax-category-carousel']")[1]
    items = hp.find_elements_by_xpath(".//li[@class='gallery-list-item height-ready']")
    for i in items:
        good = {}
        good['url'] = i.find_element_by_xpath(".//a[@class='sel-product-tile-title']").get_attribute('href')
        product_info = i.find_element_by_xpath(".//a[@class='sel-product-tile-title']").get_attribute('data-product-info')
        good['title'] = re.search(r"productName\":\s\"(.*?)\"" , product_info).group(1)
        goods.append(good)
    bclick = driver.find_element_by_xpath(".//a[@class='next-btn sel-hits-button-next'][@href]")
    bclick.click()
    time.sleep(0.5)
    
pd.DataFrame(goods)

,url,title
0,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
1,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White
2,https://www.mvideo.ru/products/naushniki-apple-airpods-w-charging-case-mv7n2ru-a-50126638,Наушники Apple AirPods w/Charging Case (MV7N2RU/A)
3,https://www.mvideo.ru/products/smartfon-samsung-galaxy-a01-black-sm-a015f-ds-30048061,Смартфон Samsung Galaxy A01 Black (SM-A015F/DS)
4,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
5,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White
6,https://www.mvideo.ru/products/naushniki-apple-airpods-w-charging-case-mv7n2ru-a-50126638,Наушники Apple AirPods w/Charging Case (MV7N2RU/A)
7,https://www.mvideo.ru/products/smartfon-samsung-galaxy-a01-black-sm-a015f-ds-30048061,Смартфон Samsung Galaxy A01 Black (SM-A015F/DS)
8,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
9,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White


In [127]:
to_dataset('goods', 'mvideo', goods)

,_id,url,title
0,5f0ba34f7617d0add6c74483,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
1,5f0ba34f7617d0add6c74484,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White
2,5f0ba34f7617d0add6c74485,https://www.mvideo.ru/products/naushniki-apple-airpods-w-charging-case-mv7n2ru-a-50126638,Наушники Apple AirPods w/Charging Case (MV7N2RU/A)
3,5f0ba34f7617d0add6c74486,https://www.mvideo.ru/products/smartfon-samsung-galaxy-a01-black-sm-a015f-ds-30048061,Смартфон Samsung Galaxy A01 Black (SM-A015F/DS)
4,5f0ba34f7617d0add6c74487,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
5,5f0ba34f7617d0add6c74488,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White
6,5f0ba34f7617d0add6c74489,https://www.mvideo.ru/products/naushniki-apple-airpods-w-charging-case-mv7n2ru-a-50126638,Наушники Apple AirPods w/Charging Case (MV7N2RU/A)
7,5f0ba34f7617d0add6c7448a,https://www.mvideo.ru/products/smartfon-samsung-galaxy-a01-black-sm-a015f-ds-30048061,Смартфон Samsung Galaxy A01 Black (SM-A015F/DS)
8,5f0ba34f7617d0add6c7448b,https://www.mvideo.ru/products/split-sistema-haier-hsu-09htm03-r2-20041878,Сплит-система Haier HSU-09HTM03/R2
9,5f0ba34f7617d0add6c7448c,https://www.mvideo.ru/products/aksessuar-dlya-kondicionera-vobix-451-white-50117777,Аксессуар для кондиционера Vobix 451 White
